# 第二回レポート課題
## 1. 2層ニューラルネットワークの学習を試してみましょう。

In [5]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import *


class TwoLayerNet:
    
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size, 
                 weight_init_std = 0.01) -> None:
        '''
        Args:
            input_size      : 入力層のニューロン数
            hidden_size     : 隠れ層のニューロン数
            output_size     : 出力層のニューロン数
        Returns:
            None
        '''
        # 重みの初期化
        self.params = {}
        
        # 一層目のweight, bias
        self.params["w1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        
        # 2層目のweight, bias
        self.params["w2"] = weight_init_std * np.random.randn(hidden_size, output_size, )
        self.params["b2"] = np.zeros(output_size)

    
    def predict(self, x) -> np.ndarray:
        '''
        推論処理
        
        Args:
            x   : 画像データ
        Returns:
            y   : ndarray
        '''
        w1 = self.params["w1"]
        w2 = self.params["w2"]
        
        b1 = self.params["b1"]
        b2 = self.params["b2"]
        
        a1 = np.dot(x, w1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, w2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        '''
        損失関数の値を求める
        
        Args:
            x : 画像データ
            t : 正解ラベル
        Returns:
            y   : ndarray
        '''
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        '''
        認識精度を求める
        
        Args:
            x : 画像データ
            t : 正解ラベル
        Returns:
            y   : ndarray
        '''
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        print(type(accuracy))
        return accuracy
    
    def numerical_gradient(self, x, t) -> dict:
        '''
        重みパラメータに対しての勾配を求める
        これは数値微分によってパラメータの勾配を求めている、おそい
        
        Args:
            x : 画像データ
            t : 正解ラベル
        Returns:
            grads : dict
        '''
        loss_w = lambda W: self.loss(x, t)
        
        grads = {}
        # 一層目のweight, biasの勾配
        grads['w1'] = numerical_gradient(loss_w, self.params['w1'])
        grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
        # 二層目のweight, biasの勾配
        grads['w2'] = numerical_gradient(loss_w, self.params['w2'])
        grads['b2'] = numerical_gradient(loss_w, self.params['b2'])
        
        return grads

In [6]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
net.params['w1'].shape
net.params['b1'].shape
net.params['w2'].shape
net.params['b2'].shape

# params変数にはネットワークに必要なパラメータがすべて格納されている
# params変数に格納された重みパラメータが推論処理(フォワード処理に使われる)

x = np.random.rand(100, 784)    # ダミーの入力データ
y = net.predict(x)              # 推論処理

In [7]:
# grads変数にはparams変数と対応するように各パラメータの勾配が格納される。

x = np.random.rand(100, 784)    # ダミーの入力データ
t = np.random.rand(100, 10)     # ダミーの正解ラベル

grads = net.numerical_gradient(x, t)    # 勾配の計算

grads['w1'].shape
grads['b1'].shape
grads['w2'].shape
grads['b2'].shape

(10,)

## ミニバッチの実装,  テストデータの評価
ニューラルネットワークの学習では、訓練データ以外のデータを正しく認識できるかどうかを確認する必要がある。これは過学習を起こしていないかの確認である。\
過学習を起こすとは訓練データに含まれない数字画像は認識できない、ということを意味する。\
エポック(epoch)は単位を表す。1エポックは学習において訓練データをすべて使い切った時の回数に対応する。\
例えば1000個の訓練データに対して100個のミニバッチで学習する場合、確率的勾配降下法を100回繰り返したら、すべての訓練データを見たことになるので、この場合100回=1エポックとなる

In [ ]:
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

# ハイパーパラメータ
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

# テストデータ評価
train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1エポック当たりの繰り返し数
iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # ミニバッチの取得
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 勾配の計算
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # パラメータの更新
    for key in ('w1', 'b1', 'w2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 学習経過の記録
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1エポックごとにすべての訓練データとテストデータに対して認識精度を計算して、結果を記録する。
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test_acc | " + str(train_acc) + ", " + str(test_acc))
    
import matplotlib.pyplot as plt
# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()